In [1]:
## 2.1.1. Environment preparation
from google.cloud import aiplatform 
aiplatform.init()

In [8]:
## VARS
# 2.1.2. Prediction container
serving_container_image_uri = "europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest" # Prebuilt container image URI - No GPU

# Model Registry
gcs_artifact_uri = "gs://iris_demo/saved_model_iris"
location = "europe-west1"
model_display_name = "online_inf_demo_iris"
# Endpoint variables
endpoint_id_prefix="projects/606366714896/locations/" + location + "/endpoints/"
endpoint_id="4"
display_name="demo_online_inf"
ep_id_uri = endpoint_id_prefix + endpoint_id

In [5]:
# 2.1.3. Upload model from GCS to Vertex AI
model = aiplatform.Model.upload(
    serving_container_image_uri=serving_container_image_uri,
    artifact_uri=gcs_artifact_uri,
    location=location,
    display_name=model_display_name
)

Creating Model
Create Model backing LRO: projects/606366714896/locations/europe-west1/models/163695291143290880/operations/570173744815800320
Model created. Resource name: projects/606366714896/locations/europe-west1/models/163695291143290880
To use this Model in another session:
model = aiplatform.Model('projects/606366714896/locations/europe-west1/models/163695291143290880')


In [7]:
print(model.resource_name)

projects/606366714896/locations/europe-west1/models/163695291143290880


In [10]:
# 2.1.4. Creating endpoint
try:
    endpoint = aiplatform.Endpoint.create(
        endpoint_id=endpoint_id,
        location=location, 
        display_name=display_name
    )
except:
    endpoint = aiplatform.Endpoint(endpoint_name = endpoint_id_prefix + endpoint_id)
    print("Endpoint already exists")


Endpoint already exists


In [11]:
print(endpoint.resource_name)

projects/606366714896/locations/europe-west1/endpoints/4


In [19]:
# 2.1.5. Deploying the model from Model Registry to an endpoint
model = aiplatform.Model(model_name=model.resource_name)
endpoint = aiplatform.Endpoint(endpoint_name=endpoint.resource_name)

model.deploy(
    endpoint=endpoint
)

Deploying model to Endpoint : projects/606366714896/locations/europe-west1/endpoints/3
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/606366714896/locations/europe-west1/endpoints/3/operations/3880319470933114880
Endpoint model deployed. Resource name: projects/606366714896/locations/europe-west1/endpoints/3


resource name: projects/606366714896/locations/europe-west1/endpoints/3

In [20]:
endpoint = aiplatform.Endpoint(endpoint_name="projects/606366714896/locations/europe-west1/endpoints/4")

In [24]:
example_instance=[{
    'sepal_length': 4.5, 
    'sepal_width': 2.3, 
    'petal_length': 1.3, 
    'petal_width':0.3
}]

In [26]:
example_instance=[
    {
    'sepal_length': 4.5, 
    'sepal_width': 2.3, 
    'petal_length': 1.3, 
    'petal_width':0.3
    },
    {
    'sepal_length': 4.0, 
    'sepal_width': 3.3, 
    'petal_length': 1.4, 
    'petal_width':0.2
    }
]
    

In [27]:
print(endpoint.predict(instances=example_instance))

Prediction(predictions=[[0.460576326], [0.476884604]], deployed_model_id='4586511600197828608', explanations=None)


In [15]:
print(endpoint.predict([{'sepal_length':5.1, 'sepal_width':2.5, 'petal_length':3.0,'petal_width':1.1}]))

Prediction(predictions=[[0.403363079]], deployed_model_id='6849570412951502848', explanations=None)
